In [1]:
from IPython.display import display, Markdown, Latex

# Play with pulp library and select squad with constraints using linear programming

In [2]:
import pandas as pd
import pulp
import numpy as np
import os

In [3]:
data_path = os.path.dirname(os.getcwd()) + '\\data\\raw\\Fantasy-Premier-League\\'

data_2021 = pd.read_csv(data_path + '2021-22/gws/merged_gw.csv')
data_2021.tail()

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
18732,Kurt Zouma,DEF,West Ham,2.6,0,0,6,0,1.8,128,...,3,0.0,-1,10796,12551,1755,54,False,0,30
18733,Aaron Cresswell,DEF,West Ham,3.8,0,0,15,0,14.8,411,...,3,4.0,1,18750,35570,16820,54,False,0,30
18734,John Ruddy,GK,Wolves,0.5,0,0,0,0,0.0,452,...,2,0.0,0,63,170,107,43,True,0,30
18735,Wilfred Ndidi,MID,Leicester,0.5,0,0,0,0,0.0,216,...,2,0.0,0,-207,538,745,48,True,0,30
18736,Ryan Fredericks,DEF,West Ham,0.2,0,0,0,0,0.0,415,...,3,0.0,0,81,139,58,44,False,0,30


# Create solver on whole season data

In [4]:
# data_2021 grouped by name and sum of 'total_points'
gw_sum = data_2021[['name', 'team', 'position', 'total_points', 'value']]
gw_sum = gw_sum.groupby(['name', 'team', 'position']).agg({'value': 'max', 'total_points': 'sum'}).reset_index()
gw_sum['value'] = gw_sum['value'] / 10
gw_sum['position'] = gw_sum['position'].replace({'GK': 1, 'DEF': 2, 'MID': 3, 'FWD': 4})
gw_sum.sort_values(by='total_points', ascending=False).head(10)

,name,team,position,value,total_points
502,Mohamed Salah,Liverpool,3,13.3,222
683,Trent Alexander-Arnold,Liverpool,2,8.5,187
264,Heung-Min Son,Spurs,3,10.9,165
368,João Pedro Cavaco Cancelo,Man City,2,7.2,162
317,Jarrod Bowen,West Ham,3,7.1,153
695,Virgil van Dijk,Liverpool,2,6.8,150
40,Andrew Robertson,Liverpool,2,7.2,142
620,Sadio Mané,Liverpool,3,12.0,140
29,Alisson Ramses Becker,Liverpool,1,6.0,138
103,Bukayo Saka,Arsenal,3,6.7,138


In [5]:
def select_team(expected_scores, prices, positions, clubs, total_budget=100, sub_factor=0.15):
    num_players = len(expected_scores)
    model = pulp.LpProblem("Constrained value maximisation", pulp.LpMaximize)
    decisions = [
        pulp.LpVariable("x{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]
    captain_decisions = [
        pulp.LpVariable("y{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]
    sub_decisions = [
        pulp.LpVariable("z{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]


    # objective function:
    model += sum((captain_decisions[i] + decisions[i] + sub_decisions[i]*sub_factor) * expected_scores[i]
                 for i in range(num_players)), "Objective"

    # cost constraint
    model += sum((decisions[i] + sub_decisions[i]) * prices[i] for i in range(num_players)) <= total_budget  # total cost

    # position constraints
    # 1 starting goalkeeper
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 1) == 1
    # 2 total goalkeepers
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 1) == 2

    # 3-5 starting defenders
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 2) >= 3
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 2) <= 5
    # 5 total defenders
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 2) == 5

    # 3-5 starting midfielders
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 3) >= 3
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 3) <= 5
    # 5 total midfielders
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 3) == 5

    # 1-3 starting attackers
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 4) >= 1
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 4) <= 3
    # 3 total attackers
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 4) == 3

    # club constraint
    for club_id in np.unique(clubs):
        model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if clubs[i] == club_id) <= 3  # max 3 players

    model += sum(decisions) == 11  # total team size
    model += sum(captain_decisions) == 1  # 1 captain

    for i in range(num_players):
        model += (decisions[i] - captain_decisions[i]) >= 0  # captain must also be on team
        model += (decisions[i] + sub_decisions[i]) <= 1  # subs must not be on team

    model.solve()
    print("Total expected score = {}".format(model.objective.value()))

    return decisions, captain_decisions, sub_decisions

In [6]:
import pulp
import numpy as np

position_data = {
    "gk": {"position_id": 1, "min_starters": 1, "max_starters": 1, "num_total": 2},
    "df": {"position_id": 2, "min_starters": 3, "max_starters": 5, "num_total": 5},
    "mf": {"position_id": 3, "min_starters": 3, "max_starters": 5, "num_total": 5},
    "fw": {"position_id": 4, "min_starters": 1, "max_starters": 3, "num_total": 3},
}


def get_decision_array(name, length):
    return np.array([
        pulp.LpVariable("{}_{}".format(name, i), lowBound=0, upBound=1, cat='Integer')
        for i in range(length)
    ])


class TransferOptimiser:
    def __init__(self, expected_scores, buy_prices, sell_prices, positions, clubs):
        self.expected_scores = expected_scores
        self.buy_prices = buy_prices
        self.sell_prices = sell_prices
        self.positions = positions
        self.clubs = clubs
        self.num_players = len(buy_prices)

    def instantiate_decision_arrays(self):
        # we will make transfers in and out of the squad, and then pick subs and captains from that squad
        transfer_in_decisions_free = get_decision_array("transfer_in_free", self.num_players)
        transfer_in_decisions_paid = get_decision_array("transfer_in_paid", self.num_players)
        transfer_out_decisions = get_decision_array("transfer_out_paid", self.num_players)
        # total transfers in will be useful later
        transfer_in_decisions = transfer_in_decisions_free + transfer_in_decisions_paid

        sub_decisions = get_decision_array("subs", self.num_players)
        captain_decisions = get_decision_array("captain", self.num_players)
        return transfer_in_decisions_free, transfer_in_decisions_paid, transfer_out_decisions, transfer_in_decisions, sub_decisions, captain_decisions

    def encode_player_indices(self, indices):
        decisions = np.zeros(self.num_players)
        decisions[indices] = 1
        return decisions

    def apply_transfer_constraints(self, model, transfer_in_decisions_free, transfer_in_decisions,
                                   transfer_out_decisions, budget_now):
        # only 1 free transfer
        model += sum(transfer_in_decisions_free) <= 1

        # budget constraint
        transfer_in_cost = sum(transfer_in_decisions * self.buy_prices)
        transfer_out_cost = sum(transfer_out_decisions * self.sell_prices)
        budget_next_week = budget_now + transfer_out_cost - transfer_in_cost
        model += budget_next_week >= 0


    def solve(self, current_squad_indices, budget_now, sub_factor):
        current_squad_decisions = self.encode_player_indices(current_squad_indices)

        model = pulp.LpProblem("Transfer optimisation", pulp.LpMaximize)
        transfer_in_decisions_free, transfer_in_decisions_paid, transfer_out_decisions, transfer_in_decisions, sub_decisions, captain_decisions = self.instantiate_decision_arrays()

        # calculate new team from current team + transfers
        next_week_squad = current_squad_decisions + transfer_in_decisions - transfer_out_decisions
        starters = next_week_squad - sub_decisions

        # points penalty for additional transfers
        transfer_penalty = sum(transfer_in_decisions_paid) * 4

        self.apply_transfer_constraints(model, transfer_in_decisions_free, transfer_in_decisions,
                                        transfer_out_decisions, budget_now)
        self.apply_formation_constraints(model, squad=next_week_squad, starters=starters,
                                         subs=sub_decisions, captains=captain_decisions)

        # objective function:
        model += self.get_objective(starters, sub_decisions, captain_decisions, sub_factor, transfer_penalty, self.expected_scores), "Objective"
        status = model.solve()

        print("Solver status: {}".format(status))

        return transfer_in_decisions, transfer_out_decisions, starters, sub_decisions, captain_decisions

    def get_objective(self, starters, subs, captains, sub_factor, transfer_penalty, scores):
        starter_points = sum(starters * scores)
        sub_points = sum(subs * scores) * sub_factor
        captain_points = sum(captains * scores)
        return starter_points + sub_points + captain_points - transfer_penalty

    def apply_formation_constraints(self, model, squad, starters, subs, captains):
        for position, data in position_data.items():
            # formation constraints
            model += sum(starter for starter, position in zip(starters, self.positions) if position == data["position_id"]) >= data["min_starters"]
            model += sum(starter for starter, position in zip(starters, self.positions) if position == data["position_id"]) <= data["max_starters"]
            model += sum(selected for selected, position in zip(squad, self.positions) if position == data["position_id"]) == data["num_total"]

        # club constraint
        for club_id in np.unique(self.clubs):
            model += sum(selected for selected, club in zip(squad, self.clubs) if club == club_id) <= 3  # max 3 players

        # total team size
        model += sum(starters) == 11
        model += sum(squad) == 15
        model += sum(captains) == 1

        for i in range(self.num_players):
            model += (starters[i] - captains[i]) >= 0  # captain must also be on team
            model += (starters[i] + subs[i]) <= 1  # subs must not be on team

In [7]:
decisions, captain_decisions, sub_decisions = select_team(gw_sum.total_points.values, gw_sum.value.values,
                                                          gw_sum.position.values, gw_sum.team.values, sub_factor=1)

player_indices = []

# print results
for i in range(gw_sum.shape[0]):
    if decisions[i].value() != 0:
        display(
            Markdown("**{}** Points = {}, Price = {}".format(gw_sum.name[i], gw_sum.total_points[i], gw_sum.value[i])))
        player_indices.append(i)
print()
print("Subs:")
# print results
for i in range(gw_sum.shape[0]):
    if sub_decisions[i].value() == 1:
        display(
            Markdown("**{}** Points = {}, Price = {}".format(gw_sum.name[i], gw_sum.total_points[i], gw_sum.value[i])))
        player_indices.append(i)

print()
print("Captain:")
# print results
for i in range(gw_sum.shape[0]):
    if captain_decisions[i].value() == 1:
        display(
            Markdown("**{}** Points = {}, Price = {}".format(gw_sum.name[i], gw_sum.total_points[i], gw_sum.value[i])))

d:\studies\informatyka\semestr_3\fpl_ai_manager\venv\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Total expected score = 2265.0


**Alisson Ramses Becker** Points = 138, Price = 6.0

**Aymeric Laporte** Points = 123, Price = 5.8

**Bukayo Saka** Points = 138, Price = 6.7

...

**Ivan Toney** Points = 109, Price = 6.7

**José Malheiro de Sá** Points = 131, Price = 5.3

**Trent Alexander-Arnold** Points = 187, Price = 8.5


Captain:


**Mohamed Salah** Points = 222, Price = 13.3

In [8]:
player_indices

[29, 65, 103, 138, 207, 288, 317, 368, 479, 502, 660, 136, 280, 366, 683]

## Transfer Optimiser

In [23]:
opt = TransferOptimiser(gw_sum.total_points.values, gw_sum.value.values, gw_sum.value.values, gw_sum.position.values, gw_sum.team.values)

transfer_in_decisions, transfer_out_decisions, starters, sub_decisions, captain_decisions = opt.solve(player_indices, budget_now=0.5, sub_factor=0.15)

Solver status: 1


In [24]:
for i in range(len(transfer_in_decisions)):
    if transfer_in_decisions[i].value() == 1:
        print("Transferred in: {} {} {}".format(gw_sum.name[i], gw_sum.total_points[i], gw_sum.value[i]))
    if transfer_out_decisions[i].value() == 1:
        print("Transferred out: {} {} {}".format(gw_sum.name[i], gw_sum.total_points[i], gw_sum.value[i]))

Transferred in: Aaron Ramsdale 117 5.2
Transferred out: Alisson Ramses Becker 138 6.0
Transferred out: Aymeric Laporte 123 5.8
Transferred in: Virgil van Dijk 150 6.8
